In [1]:
# dependencies import
from common_dependencies import *
import logging
logger = logging.getLogger('main.(02-05)v_make_pred.ipynb')

2024-03-13 17:37:29.790167: E external/local_xla/xla/stream_executor/plugin_registry.cc:91] Invalid plugin kind specified: FFT
2024-03-13 17:37:30.301831: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 17:37:30.897780: E external/local_xla/xla/stream_executor/plugin_registry.cc:91] Invalid plugin kind specified: DNN


In [2]:
# paths for files with original data
PATH_TO_DATA = {
    'run_1': 
        ('data/original_data/run_1/run1_WM32_data.csv',
        'data/original_data/run_1/run1_WM32_defects.csv',
        'data/original_data/run_1/run1_WM32_pipe.csv'),
    'run_2':
        ('data/original_data/run_2/run2_WM32_data.csv',
        'data/original_data/run_2/run2_WM32_defects.csv',
        'data/original_data/run_2/run2_WM32_pipe.csv')
}

def get_quadratic_sequence(max_val: int, res_list: list[int] = [4]) -> list[int]:
    if max_val % 2 == 1:
        raise ValueError('max_val should be integer divisible by 2')
        
    if max(res_list) == max_val:
        return res_list
    res_list.append(max(res_list) * 2)
    return get_quadratic_sequence(max_val, res_list)


PATH_TO_MODEL = 'networks/CNN'
RUNS = [1, 2]
MODEL_VER = '06'
MODEL_NUM = '03'
XSHIFT = 200

for name in os.listdir(PATH_TO_MODEL):
    res = re.match(F'(id=v{MODEL_VER}n{MODEL_NUM}).*', name)
    if not res is None:
        PATH_TO_MODEL += '/' + res[0]
        break

# детерминация случайных величин, отвечающих за выбор первоначальных весов и биасов
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)

In [3]:
# Загрузка модели
load_model = keras.models.load_model(PATH_TO_MODEL)
CROP_SIZE = load_model.inputs[0].shape[1]
CROP_STEPS = get_quadratic_sequence(CROP_SIZE)

2024-03-13 17:37:32.822618: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:924] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-13 17:37:33.166235: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:924] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-13 17:37:33.166295: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:924] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-13 17:37:33.167541: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:924] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-13 17:37:33.167623: I external/local_xla/xla/stream_executor/rocm/rocm_executor.

In [4]:
def make_pred_fast(model, 
                   path_to_data_tuple: tuple(),
                   x_shift: int,
                   crop_size: int, 
                   crop_step: int):

    # read data and roll dfs along x axis
    x_df, y_df = dw.get_x_and_y_data(*path_to_data_tuple)
    x_df = dw.roll_df(x_df, x_shift, axis=1)
    y_df = dw.roll_df(y_df, x_shift, axis=1)

    # save the read df shape
    or_rows = x_df.shape[0]
    or_cols = x_df.shape[1]

    # extend x and y dfs for prediction and crops dividing
    ex_x_df = dw.extend_df_for_prediction(x_df, crop_size)
    ex_x_df = dw.extend_df_for_crops_dividing(ex_x_df, crop_size, crop_step)
    ex_y_df = dw.extend_df_for_prediction(y_df, crop_size)
    ex_y_df = dw.extend_df_for_crops_dividing(ex_y_df, crop_size, crop_step)

    ex_rows = ex_x_df.shape[0]
    ex_cols = ex_x_df.shape[1]

    # reshape x df to set of crops
    (x_data_time,
    x_data_amp) = dw.reshape_x_df_to_image_like_numpy(ex_x_df, crop_size, crop_step)

    # standardize x data
    x_data_time = dw.standardize_data(x_data_time)
    x_data_amp = dw.standardize_data(x_data_amp)
    
    res = np.array(model.predict([x_data_time, x_data_amp]))
    res = np.squeeze(res, axis=2)
    
    bool_res = res[0,:]
    depth_res = res[1,:]

    extend_size_arr = np.ones((ex_rows, ex_cols))     
    
    bool_res_it = iter(bool_res) 
    depth_res_it = iter(depth_res) 
    
    for j in range(0,  ex_cols - crop_size + 1, crop_step):
        for i in range(0, ex_rows - crop_size + 1, crop_step):  
            bool_temp_add = next(bool_res_it)
            depth_temp_add = next(depth_res_it)
            extend_size_arr[i:i+crop_size, j:j+crop_size] += bool_temp_add * depth_temp_add
    
    orig_size_arr = extend_size_arr[crop_size - 1:, crop_size - 1:][:or_rows, :or_cols]

    extend_size_result_df = pd.DataFrame(data=extend_size_arr, 
                             columns=ex_x_df.columns, 
                             index=ex_x_df.index)

    orig_size_result_df = pd.DataFrame(data=orig_size_arr, 
                       columns=x_df.columns.tolist(), 
                       index=x_df.index.tolist())
    
    return orig_size_result_df, extend_size_result_df, y_df, ex_y_df 

In [5]:
for run in RUNS:
    for crop_step in CROP_STEPS:
        logger.debug(f'run: {run}, crop step: {crop_step}')
        (orig_size_result_df, 
        extend_size_result_df,
        orig_size_reference_df, 
        extend_size_reference_df) = make_pred_fast(load_model, PATH_TO_DATA[f'run_{run}'], XSHIFT, CROP_SIZE, crop_step)

        path_to_run = f'data/drawing_data/model_id=v{MODEL_VER}n{MODEL_NUM}/run_{run}/' 
        res_file_name = f'model_id=v{MODEL_VER}n{MODEL_NUM}_crop(size={CROP_SIZE},step={crop_step})_shift(x={XSHIFT},y=0).xlsx'
        
        if not os.path.exists(path_to_run):
            os.makedirs(path_to_run)
        
        with pd.ExcelWriter(os.path.join(path_to_run, res_file_name)) as writer:  
            orig_size_result_df.to_excel(writer, sheet_name='orig_size_result')
            extend_size_result_df.to_excel(writer, sheet_name='extend_size_result')
            
            orig_size_reference_df.to_excel(writer, sheet_name='orig_size_reference')
            extend_size_reference_df.to_excel(writer, sheet_name='extend_size_reference')

main.(02-05)v_make_pred.ipynb ::             <module>() :: run: 1, crop step: 4
DEBUG:main.(02-05)v_make_pred.ipynb:run: 1, crop step: 4
main.custom_modules.data_worker.data_worker ::     get_x_and_y_data() :: 
    Read detectors data shape: (115, 400)
    Read defect data shape: (115, 400)
DEBUG:main.custom_modules.data_worker.data_worker:
    Read detectors data shape: (115, 400)
    Read defect data shape: (115, 400)
main.custom_modules.data_worker._dataframe_utils :: extend_df_for_prediction() :: 
    The input df shape: (145, 430)
    The crop size: 16
    The output df shape: (145, 430)
DEBUG:main.custom_modules.data_worker._dataframe_utils:
    The input df shape: (145, 430)
    The crop size: 16
    The output df shape: (145, 430)
main.custom_modules.data_worker._dataframe_utils :: extend_df_for_crops_dividing() :: 
    The input df shape: (148, 432)
    The crop size: 16
    The crop step: 4
    The output df shape: (148, 432)
DEBUG:main.custom_modules.data_worker._dataframe_u

 49/112 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

I0000 00:00:1710326263.378100   11277 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


112/112 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step


main.(02-05)v_make_pred.ipynb ::             <module>() :: run: 1, crop step: 8
DEBUG:main.(02-05)v_make_pred.ipynb:run: 1, crop step: 8
main.custom_modules.data_worker.data_worker ::     get_x_and_y_data() :: 
    Read detectors data shape: (115, 400)
    Read defect data shape: (115, 400)
DEBUG:main.custom_modules.data_worker.data_worker:
    Read detectors data shape: (115, 400)
    Read defect data shape: (115, 400)
main.custom_modules.data_worker._dataframe_utils :: extend_df_for_prediction() :: 
    The input df shape: (145, 430)
    The crop size: 16
    The output df shape: (145, 430)
DEBUG:main.custom_modules.data_worker._dataframe_utils:
    The input df shape: (145, 430)
    The crop size: 16
    The output df shape: (145, 430)
main.custom_modules.data_worker._dataframe_utils :: extend_df_for_crops_dividing() :: 
    The input df shape: (152, 432)
    The crop size: 16
    The crop step: 8
    The output df shape: (152, 432)
DEBUG:main.custom_modules.data_worker._dataframe_u

...

115/115 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step


main.(02-05)v_make_pred.ipynb ::             <module>() :: run: 2, crop step: 8
DEBUG:main.(02-05)v_make_pred.ipynb:run: 2, crop step: 8
main.custom_modules.data_worker.data_worker ::     get_x_and_y_data() :: 
    Read detectors data shape: (119, 400)
    Read defect data shape: (119, 400)
DEBUG:main.custom_modules.data_worker.data_worker:
    Read detectors data shape: (119, 400)
    Read defect data shape: (119, 400)
main.custom_modules.data_worker._dataframe_utils :: extend_df_for_prediction() :: 
    The input df shape: (149, 430)
    The crop size: 16
    The output df shape: (149, 430)
DEBUG:main.custom_modules.data_worker._dataframe_utils:
    The input df shape: (149, 430)
    The crop size: 16
    The output df shape: (149, 430)
main.custom_modules.data_worker._dataframe_utils :: extend_df_for_crops_dividing() :: 
    The input df shape: (152, 432)
    The crop size: 16
    The crop step: 8
    The output df shape: (152, 432)
DEBUG:main.custom_modules.data_worker._dataframe_u

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


main.(02-05)v_make_pred.ipynb ::             <module>() :: run: 2, crop step: 16
DEBUG:main.(02-05)v_make_pred.ipynb:run: 2, crop step: 16
main.custom_modules.data_worker.data_worker ::     get_x_and_y_data() :: 
    Read detectors data shape: (119, 400)
    Read defect data shape: (119, 400)
DEBUG:main.custom_modules.data_worker.data_worker:
    Read detectors data shape: (119, 400)
    Read defect data shape: (119, 400)
main.custom_modules.data_worker._dataframe_utils :: extend_df_for_prediction() :: 
    The input df shape: (149, 430)
    The crop size: 16
    The output df shape: (149, 430)
DEBUG:main.custom_modules.data_worker._dataframe_utils:
    The input df shape: (149, 430)
    The crop size: 16
    The output df shape: (149, 430)
main.custom_modules.data_worker._dataframe_utils :: extend_df_for_crops_dividing() :: 
    The input df shape: (160, 432)
    The crop size: 16
    The crop step: 16
    The output df shape: (160, 432)
DEBUG:main.custom_modules.data_worker._datafram

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
